In [1]:
from DELCgen import *
import scipy.stats as st
import matplotlib.pyplot as plt 
from constants import *
from scipy.interpolate import interp1d
import naima
import astropy.units as u
import subroutines as sub
import os

This piece of code runs a simple simulation where a CR population is injected and evolved with a variable jet. We need to define the maximum rigidity which is related to the jet power
$$
R_{\mathrm{max}} = 10^{19}~V\left(\frac{Q_j}{10^{44}~\mathrm{erg~s}^{-1}}\right)^{1/2}
    \left(\frac{\eta}{0.1}\right)
    \left(\frac{u}{0.1c}\right).
$$

the "power threshold" is the inverse expression with $Q_j$ as the subject. We use an input power spectral density (PSD) and jet power probability distribution function (PDF) to generate a time series for the jet power for a given set of PSD and PDF parameters. To do this, we use the method described by Emmanolopoulous et al. and implemented in python by Connolly. The PSD is implemented as a bending power-law model of the form 

$$\mathrm{PSD}(\nu,\bar{Q},\nu_c) = A(\bar{Q}) \frac{\nu^{-\alpha_{\mathrm{low}}}}
               {1+(\nu/\nu_c)^{\alpha_{\mathrm{high}}-\alpha_{\mathrm{low}}}}$$
               
               
We set $\alpha_{\mathrm{low}} = 1$ for a pink noise spectrum, $\alpha_{\mathrm{high}} = 10$ so as to suppress high frequency variability, and $\nu_c = 1$Myr, roughly equal to the light travel time, $\tau_c=L/c$, along a $L=300$kpc jet. The acceleration time relates to $\tau_c$ by $\tau_\mathrm{acc}/\tau_c \approx (Rc)/(uL)$, since the highest energy particles reach Larmor radii comparable to $(u/c)R$ (Hillas criterion). The longest CR acceleration time is therefore shorter than the light travel time along the jet for any fast, long and thin jet ($u/c>R/L$). 

In [2]:
def power_threshold(rigidity, v_over_c=0.1, eta=0.1):
    power = (0.1 / eta) * (rigidity / 1e19)**2 * (0.1/v_over_c) * 1e44
    return power 

def max_energy(power, v_over_c=0.1, eta=0.1):
    return 1e19*np.sqrt( (power/1e44) * (v_over_c/0.1))

def get_lc(lognorm_params, PSD_params, tbin, Age):
    # Simulation params
    # let's do everything in units of kyr
    # run for 100 Myr (1e5 kyr) in bins of 0.1 Myr
    #lognorm_params = (1.5,0,np.exp(1.5))
    RedNoiseL,RandomSeed,aliasTbin = 100,12,100
    N = Age / tbin

    lc = Simulate_DE_Lightcurve(BendingPL, PSD_params,st.lognorm,lognorm_params,
                                    RedNoiseL=RedNoiseL,aliasTbin=aliasTbin,randomSeed=RandomSeed,LClength=Age, tbin=tbin)

    return (lc)

Now we've set up the basic functions. Let's initialise our parameters for the variable jet history and initialise the tau_loss dictionary which stores the loss times for the different species (calculated using CRPropa).  

In [3]:
# set up light curve. pl_index of 1 means red noise.
pl_index = 1

# bend the power law at 1 Myr (1e-3 kyr^-1) and steeply decay it beyond that with index 10. 
# A,v_bend,a_low,a_high,c = 1, 1e-3, pl_index, 10, 1
PSD_params = (1, 1e-3, pl_index, 10, 1)
lognorm_params = (1.5,0,np.exp(1.5))
tbin = 100    # 100 kyr 

# Age of 1000 means 100 Myr
# paramaters for lc are lognorm parameters, PSD parameters, tbin and Age (Age is really number of points)
lc = get_lc(lognorm_params, PSD_params, tbin, 500)

# time is in kyr, so convert to Myr
times = np.arange(0,1000.0*tbin,tbin) 

Set up arrays to loop over. flux_scales contains the normalisations of our jet power $(\bar{Q})$.
This is actually the median of the distribution, or the mean in log space.
betas is the spectral index of the injected spectrum ($\beta$).

In [4]:
# flux_scales contains the normalisations of our jet power 
# actually the median of the distribution, or the mean in log space.
# betas is the spectral index of the injected spectrum 
#flux_scales = np.logspace(43,45,num=10)
#betas = np.arange(2,3,0.1)
betas = [2,2.3,2.7]
flux_scales = [1e43,1e44]

In [5]:
energies = np.logspace(6,20.5,num=3000)
for i_flux, flux_scale in enumerate(flux_scales):

    # normalise the light curve
    flux = lc.flux * flux_scale 
    
    # loop over spectral indices
    for i_beta, BETA in enumerate(betas):
        
        # load the arrays calculated
        escaping_time = np.load("array_saves/escaping_beta{:.1f}q{:.1f}.npy".format(BETA, np.log10(flux_scale)))
        ncr_time = np.load("array_saves/ncr_beta{:.1f}q{:.1f}.npy".format(BETA, np.log10(flux_scale)))
    
        # elemental "abundances" - really injection fractions 
        # this is roughly solar, but should probably be top heavy since it is easier to inject heavy ions, generally
        frac_elem = np.array([1.0,0.1,1e-4,3.16e-05])
        
        # normalise! 
        frac_elem /= np.sum(frac_elem)
        
        # get the time step - IMPROVE 
        delta_t = tbin * 1000.0 * YR
        
        # charges 
        z_elem = np.array([1,2,7,26])
    
        lgamma = np.zeros_like(flux)
        
        movie_counter = 0
        DELTA = 3
        
        i = len(flux) -1 
        
        #for i in range(0,len(flux), DELTA):
        # get the maximum rigidity from the power requirement
        Rmax = max_energy(flux[i], v_over_c=0.5)

        # this plots the lightcurve 
        fig, ax2, ax3 = sub.init_lc_plot(lc.time[:i+1] / 1000.0, flux[:i+1])


        for j, frac in enumerate(frac_elem):

            rigidities = energies / z_elem[j]

            sub.plot_spectra(fig, ax2, ax3, rigidities * z_elem[j], ncr_time[j,i,:], escaping_time[j,i,:]*delta_t, j)


        #os.system("mkdir spectra/beta{:.1f}q{:.1f}".format(BETA, np.log10(flux_scale)))
        plt.savefig("spec_beta{:.1f}q{:.1f}.png".format(BETA, np.log10(flux_scale)), dpi=100)
        plt.close("all")
        movie_counter += 1
                
        print ("BETA {:.1f} median lum {:8.4e} mean lum {:8.4e}".format(BETA, flux_scale, np.mean(flux)))


BETA 2.0 median lum 1.0000e+43 mean lum 1.3231e+44
BETA 2.3 median lum 1.0000e+43 mean lum 1.3231e+44
BETA 2.7 median lum 1.0000e+43 mean lum 1.3231e+44
BETA 2.0 median lum 1.0000e+44 mean lum 1.3231e+45
BETA 2.3 median lum 1.0000e+44 mean lum 1.3231e+45
BETA 2.7 median lum 1.0000e+44 mean lum 1.3231e+45


Let's loop over the flux scales and the spectral indices. The method is as follows. For each value of $\bar{Q}$ and $\beta$, we evolve a CR distribution for each ionic species numerically using the following equation,

$$
    \frac{dn_i(E)}{dt} = S(E/Z_i, f_i, t) - \frac{n(E)}{\tau_\mathrm{esc} (E/Z_i)} - \frac{n(E)}{\tau_\mathrm{loss} (E, Z_i)}.
$$

The total differential CR spectrum is then just $n(E,t)=\sum_i n_i(E,t)f_i$. We follow, e.g., Eichmann et al. 2018 in using a power-law for the injection term $S({\cal R}_i, f_i, t)$ with a sharp cutoff, given by

$$
    S(E/Z_i, f_i, t) = 
    f_i \eta Q_j(t)~\nu_i
    \left(\frac{{\cal R}}{{\cal R}_0}\right)^{-\beta}~f_{\mathrm{cut}}(R_{\mathrm{max}}).
    % f_\mathrm{cut}
$$

where $\nu_i$ is the normalisation of the distribution and is given by $\nu_i=1.0/[\ln({\cal R}_\mathrm{max}(t)/{\cal R}_0]$ for $\beta=2$ and $\nu_i=(2-\beta)/[({\cal R}_\mathrm{max}/{\cal R}_0)^{(2-\beta)}-1]$ for $\beta > 2$.  We set ${\cal R}_\mathrm{max}$ according to the maximum rigidity condition, ${\cal R}_0=1\mathrm{GV}$. 

In [7]:
energies = np.logspace(6,20.5,num=3000)
for i_flux, flux_scale in enumerate(flux_scales):

    # normalise the light curve
    flux = lc.flux * flux_scale 
    
    # loop over spectral indices
    for i_beta, BETA in enumerate(betas):
        
        # load the arrays calculated
        escaping_time = np.load("array_saves/escaping_beta{:.1f}q{:.1f}.npy".format(BETA, np.log10(flux_scale)))
        ncr_time = np.load("array_saves/ncr_beta{:.1f}q{:.1f}.npy".format(BETA, np.log10(flux_scale)))
    
        # elemental "abundances" - really injection fractions 
        # this is roughly solar, but should probably be top heavy since it is easier to inject heavy ions, generally
        frac_elem = np.array([1.0,0.1,1e-4,3.16e-05])
        
        # normalise! 
        frac_elem /= np.sum(frac_elem)
        
        # get the time step - IMPROVE 
        delta_t = tbin * 1000.0 * YR
        
        # charges 
        z_elem = np.array([1,2,7,26])
    
        lgamma = np.zeros_like(flux)
        
        movie_counter = 0
        DELTA = 2
        
        for i in range(1,len(flux), DELTA):
            # get the maximum rigidity from the power requirement
            Rmax = max_energy(flux[i], v_over_c=0.5)
            
            # this plots the lightcurve 
            fig, ax2, ax3 = sub.init_lc_plot(lc.time[:i+1] / 1000.0, flux[:i+1])


            for j, frac in enumerate(frac_elem):

                rigidities = energies / z_elem[j]
                
                sub.plot_spectra(fig, ax2, ax3, rigidities * z_elem[j], ncr_time[j,i,:], escaping_time[j,i,:]*delta_t, j)
    
            
            os.system("mkdir spectra/beta{:.1f}q{:.1f}".format(BETA, np.log10(flux_scale)))
            plt.savefig("spectra/beta{:.1f}q{:.1f}/spectra{:03d}.png".format(BETA, np.log10(flux_scale), movie_counter), dpi=100)
            plt.close("all")
            movie_counter += 1
                
        
        print ("BETA {:.1f} median lum {:8.4e} mean lum {:8.4e}".format(BETA, flux_scale, np.mean(flux)))


BETA 2.0 median lum 1.0000e+43 mean lum 1.3231e+44
BETA 2.3 median lum 1.0000e+43 mean lum 1.3231e+44
BETA 2.7 median lum 1.0000e+43 mean lum 1.3231e+44
BETA 2.0 median lum 1.0000e+44 mean lum 1.3231e+45
BETA 2.3 median lum 1.0000e+44 mean lum 1.3231e+45
BETA 2.7 median lum 1.0000e+44 mean lum 1.3231e+45


Now let's make a movie with ffmpeg (pretty hacky this)

In [8]:
import os
import time
import io
import base64
from IPython.display import HTML

for i_flux, flux_scale in enumerate(flux_scales):
    for i_beta, BETA in enumerate(betas):
        print (flux_scale, BETA)
        fname = "beta{:.1f}q{:.1f}".format(BETA, np.log10(flux_scale))
        cmd = "ffmpeg -y -i spectra/{}/spectra%3d.png -r 6 movies/movie_{}.mp4".format(fname,fname)
        os.system(cmd)
        time.sleep(3)
        
# open the last movie?  
moviename = "movies/movie_{}.mp4".format(fname)
video = io.open(moviename, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

from IPython.display import Video

Video(moviename )

1e+43 2
1e+43 2.3
1e+43 2.7
1e+44 2
1e+44 2.3
1e+44 2.7
